In [1]:
with open('data/input.txt', 'r', encoding='utf-8') as f:
    shakespeare = f.read()

In [2]:
print(f'length of the input text for pretraining: {len(shakespeare)}.')
print(shakespeare[:1000])

length of the input text for pretraining: 1115394.
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak thi

In [9]:
# List all unique characters that occurs in the input text
chars = sorted(list(set(shakespeare)))
vocab_size = len(chars)
print('ALL chars:' + ''.join(chars))
print()
print(f'TOTAL CHARS NUM: {vocab_size}')

ALL chars:
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz

TOTAL CHARS NUM: 65


In [11]:
# mapping from characters to integers for encoding
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s : [stoi[c] for c in s] # Take a string and output a list of integers
decode = lambda i : ''.join([itos[c] for c in i]) # Take a list of integers and output a list of string

test_string = "hii, there"
print(encode(test_string))
print(decode(encode(test_string)))

[46, 47, 47, 6, 1, 58, 46, 43, 56, 43]
hii, there
